### Load Model

In [1]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [2]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [3]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-21')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
import cv2 
import numpy as np
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
# cap.release()

### Local + Save Missing

In [26]:
from datetime import datetime
import os

class State:
    def __init__(self, source = "pencil4.mp4") -> None:
        self.source = source
        self.min_score_thresh = .5
        self.turnpoint = 5
        
        # For missed frames
        self.missedThreshold = 5
        
    def start(self):
        
        
        self.progress = 0
        self.counter = 0
        self.detected = False
        
        # For missed frames
        ctime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        self.outpath = f'missing/{ctime}/'
        os.makedirs(self.outpath)
        self.missed = 0
        
        self.stream()
    
    def stream(self):
        # Setup capture
        cap = cv2.VideoCapture(self.source)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        while True:
            ret, frame = cap.read()
            if not ret: break
            cv2.imshow('frame',  frame)
            image_np = np.array(frame)
            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            
            detections = detect_fn(input_tensor)
            
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                        for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()
            
            boxes = detections['detection_boxes']
            # get all boxes from an array
            max_boxes_to_draw = boxes.shape[0]
            # get scores to get a threshold
            scores = detections['detection_scores']
            
            # For missed frames
            maxScore = scores.max()
            if maxScore > self.min_score_thresh:
                self.detected = True
            if self.detected and maxScore < self.min_score_thresh:
                self.missed += 1
                if self.missed%self.missedThreshold == 0:
                    ctime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                    cv2.imwrite(f'{self.outpath}/{ctime}.png', frame)
            
            # # get index with max score
            # maxIdx = np.argmax(scores)
            # print(maxIdx)
            # # iterate over all objects found
            # for i in range(min(max_boxes_to_draw, boxes.shape[0])):
            #     if scores is None or scores[i] > self.min_score_thresh:
            #         # boxes[i] is the box which will be drawn
            #         class_name = category_index[detections['detection_classes'][i]+label_id_offset]['name']
            #         print("This box is gonna get used", i, boxes[i], detections['detection_classes'][i])
            
            if boxes is not None and len(boxes) > 0:
                ymin, xmin, ymax, xmax = boxes[0]
                h = ymax - ymin
                w = xmax - xmin
                match self.progress:
                    case 0:
                        if h<w:
                            self.counter += 1
                            if self.counter >= self.turnpoint:
                                self.progress = 1
                                self.counter = 0
                                print("45-135 Degrees reached")
                    case 1:
                        if h>w:
                            self.counter += 1
                            if self.counter >= self.turnpoint:
                                self.progress = 2
                                self.counter = 0
                                print("135-225 Degrees reached")
                    case 2:
                        if h<w:
                            self.counter += 1
                            if self.counter >= self.turnpoint:
                                self.progress = 3
                                self.counter = 0
                                print("225-315 Degrees reached")
                    case 3:
                        if h>w:
                            self.counter += 1
                            if self.counter >= self.turnpoint:
                                self.progress = 0
                                self.counter = 0
                                print("315-45 Degrees reached")
                                print("Rotation Complete!")
                            
            viz_utils.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=1,
                        min_score_thresh=self.min_score_thresh,
                        agnostic_mode=False)

            cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                break
        cv2.destroyAllWindows()

In [27]:
s = State(source=0)
s.start()

45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Rotation Complete!
45-135 Degrees reached
135-225 Degrees reached
225-315 Degrees reached
315-45 Degrees reached
Ro

In [ ]:
import requests
class State:
    def __init__(self, source = "pencil4.mp4") -> None:
        self.source = source
        self.min_score_thresh = .4
        
    def start(self):
        self.progress = 0
        self.counter = 0
        self.turnpoint = 5
        self.stream()
    
    def stream(self):
        # Setup capture
        cap = cv2.VideoCapture(self.source)
        while True: 
            ret, frame = cap.read()
            if not ret: break
            # cv2.imshow('frame', frame)
            image_bytes = cv2.imencode('.png', frame)[1].tobytes()
            
            URL = "http://localhost:8001/predict/"
            response = requests.put(URL, data=image_bytes, headers={'Content-Type': 'multipart/form-data'})
            print(response.content)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                break
        cv2.destroyAllWindows()
        
s = State()
s.start()

### ZMQ Protocol

In [3]:
import zmq
import cv2
import json
context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
URL = "tcp://localhost:8002"
socket.connect(URL)

class State:
    def __init__(self, source = "pencil4.mp4") -> None:
        self.source = source
        self.min_score_thresh = .4
        
    def start(self):
        self.progress = 0
        self.counter = 0
        self.turnpoint = 5
        self.stream()
    
    def stream(self):
        # Setup capture
        cap = cv2.VideoCapture(self.source)
        while True: 
            ret, frame = cap.read()
            if not ret: break
            # cv2.imshow('frame', frame)
            image_bytes = cv2.imencode('.png', frame)[1].tobytes()
            print("Sending request...")
            socket.send(image_bytes)

            # Get the reply.
            message = socket.recv()
            data = json.loads(message.decode('utf-8'))
            print(f"Received reply {message}")
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                break
        cv2.destroyAllWindows()
        
s = State()
s.start()

Connecting to hello world server…
Sending request...
Received reply b'{"location": [0.25260958075523376, 0.3857634663581848, 0.6834648847579956, 0.48991018533706665], "class": "pencil", "score": 0.6312814950942993}'
Sending request...
Received reply b'{"location": [0.2566604018211365, 0.37821951508522034, 0.6899195909500122, 0.49776676297187805], "class": "pencil", "score": 0.7835674285888672}'
Sending request...
Received reply b'{"location": [0.26824474334716797, 0.3350699841976166, 0.6835602521896362, 0.5413943529129028], "class": "pencil", "score": 0.9999510049819946}'
Sending request...
Received reply b'{"location": [0.26885712146759033, 0.2957770526409149, 0.6631417274475098, 0.588182806968689], "class": "pencil", "score": 0.9994670748710632}'
Sending request...
Received reply b'{"location": [0.25138944387435913, 0.29404884576797485, 0.6506933569908142, 0.6274974942207336], "class": "pencil", "score": 0.9999563694000244}'
Sending request...
Received reply b'{"location": [0.2346647